In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn import metrics
import joblib

Using TensorFlow backend.


In [2]:
data = pd.read_csv("../dataset")

In [3]:
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']

In [4]:
le_proto = LabelEncoder()
x[:, 0] = le_proto.fit_transform(x[:, 0])
#joblib.dump(le_proto, '../preproc/le_proto2.joblib',protocol=1) #save label encoder weights to file
le_label = LabelEncoder()
y = le_label.fit_transform(y)
#joblib.dump(le_label, '../preproc/le_label2.joblib',protocol=1) #save label encoder weights to file
sc = StandardScaler()
x=sc.fit_transform(x)
#joblib.dump(sc, '../preproc/scaler2.joblib',protocol=1)
x = np.expand_dims(x, axis=2)

In [5]:
model = Sequential()

model.add(Conv1D(16, 2, strides=1,padding='same',activation='relu',input_shape=(8,1)))
model.add(MaxPooling1D(padding='same'))

model.add(Conv1D(32, 2, strides=1,padding='same',activation='relu'))
model.add(MaxPooling1D(padding='same'))

model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 8, 16)             48        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 16)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 32)             1056      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2, 64)             2112      
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                

In [6]:
save_checkpoint = ModelCheckpoint(
    filepath = '../trained_models/cnn1d-2loop2.h5',
    monitor = 'val_accuracy',
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto'
)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

history = model.fit(x, y, batch_size=64, validation_split=0.3,callbacks=[save_checkpoint], epochs=500,shuffle=False)

Train on 11974 samples, validate on 5132 samples
Epoch 1/500
11974/11974 [==============================] - 1s 48us/step - loss: 0.9672 - accuracy: 0.7087 - val_loss: 0.2707 - val_accuracy: 0.9458
Epoch 2/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.1680 - accuracy: 0.9590 - val_loss: 0.1204 - val_accuracy: 0.9700
Epoch 3/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.1112 - accuracy: 0.9673 - val_loss: 0.0952 - val_accuracy: 0.9719
Epoch 4/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.0951 - accuracy: 0.9698 - val_loss: 0.0837 - val_accuracy: 0.9749
Epoch 5/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0863 - accuracy: 0.9710 - val_loss: 0.0767 - val_accuracy: 0.9768
Epoch 6/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0803 - accuracy: 0.9729 - val_loss: 0.0717 - val_accuracy: 0.9776
Epoch 7/500
11974/11974 [==============================] - 0s 2

Epoch 56/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0334 - accuracy: 0.9857 - val_loss: 0.0322 - val_accuracy: 0.9871
Epoch 57/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0332 - accuracy: 0.9856 - val_loss: 0.0320 - val_accuracy: 0.9869
Epoch 58/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0329 - accuracy: 0.9856 - val_loss: 0.0321 - val_accuracy: 0.9871
Epoch 59/500
11974/11974 [==============================] - 0s 30us/step - loss: 0.0327 - accuracy: 0.9858 - val_loss: 0.0317 - val_accuracy: 0.9866
Epoch 60/500
11974/11974 [==============================] - 0s 31us/step - loss: 0.0325 - accuracy: 0.9854 - val_loss: 0.0319 - val_accuracy: 0.9871
Epoch 61/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0323 - accuracy: 0.9860 - val_loss: 0.0313 - val_accuracy: 0.9867
Epoch 62/500
11974/11974 [==============================] - 0s 30us/step - loss: 0.0321 - accuracy: 0.9861

Epoch 111/500
11974/11974 [==============================] - 0s 32us/step - loss: 0.0246 - accuracy: 0.9895 - val_loss: 0.0249 - val_accuracy: 0.9873
Epoch 112/500
11974/11974 [==============================] - 0s 30us/step - loss: 0.0247 - accuracy: 0.9898 - val_loss: 0.0247 - val_accuracy: 0.9873
Epoch 113/500
11974/11974 [==============================] - 0s 33us/step - loss: 0.0244 - accuracy: 0.9898 - val_loss: 0.0246 - val_accuracy: 0.9873
Epoch 114/500
11974/11974 [==============================] - 0s 33us/step - loss: 0.0243 - accuracy: 0.9900 - val_loss: 0.0243 - val_accuracy: 0.9875
Epoch 115/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0241 - accuracy: 0.9901 - val_loss: 0.0249 - val_accuracy: 0.9871
Epoch 116/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0241 - accuracy: 0.9901 - val_loss: 0.0240 - val_accuracy: 0.9877
Epoch 117/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0239 - accuracy:

Epoch 220/500
11974/11974 [==============================] - 0s 34us/step - loss: 0.0150 - accuracy: 0.9951 - val_loss: 0.0199 - val_accuracy: 0.9918
Epoch 221/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0149 - accuracy: 0.9948 - val_loss: 0.0197 - val_accuracy: 0.9918
Epoch 222/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0150 - accuracy: 0.9948 - val_loss: 0.0199 - val_accuracy: 0.9916
Epoch 223/500
11974/11974 [==============================] - 0s 33us/step - loss: 0.0149 - accuracy: 0.9949 - val_loss: 0.0197 - val_accuracy: 0.9918
Epoch 224/500
11974/11974 [==============================] - 0s 32us/step - loss: 0.0148 - accuracy: 0.9949 - val_loss: 0.0199 - val_accuracy: 0.9918
Epoch 225/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0148 - accuracy: 0.9949 - val_loss: 0.0196 - val_accuracy: 0.9918
Epoch 226/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.0147 - accuracy:

Epoch 329/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0130 - accuracy: 0.9949 - val_loss: 0.0169 - val_accuracy: 0.9916
Epoch 330/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0130 - accuracy: 0.9949 - val_loss: 0.0165 - val_accuracy: 0.9920
Epoch 331/500
11974/11974 [==============================] - 0s 31us/step - loss: 0.0132 - accuracy: 0.9952 - val_loss: 0.0164 - val_accuracy: 0.9920
Epoch 332/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0131 - accuracy: 0.9951 - val_loss: 0.0178 - val_accuracy: 0.9916
Epoch 333/500
11974/11974 [==============================] - 0s 30us/step - loss: 0.0130 - accuracy: 0.9951 - val_loss: 0.0164 - val_accuracy: 0.9922
Epoch 334/500
11974/11974 [==============================] - 0s 32us/step - loss: 0.0132 - accuracy: 0.9950 - val_loss: 0.0184 - val_accuracy: 0.9916
Epoch 335/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.0130 - accuracy:

Epoch 438/500
11974/11974 [==============================] - 0s 27us/step - loss: 0.0109 - accuracy: 0.9959 - val_loss: 0.0206 - val_accuracy: 0.9920
Epoch 439/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0111 - accuracy: 0.9955 - val_loss: 0.0220 - val_accuracy: 0.9908
Epoch 440/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0108 - accuracy: 0.9960 - val_loss: 0.0199 - val_accuracy: 0.9922
Epoch 441/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0106 - accuracy: 0.9962 - val_loss: 0.0206 - val_accuracy: 0.9914
Epoch 442/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0108 - accuracy: 0.9959 - val_loss: 0.0200 - val_accuracy: 0.9918
Epoch 443/500
11974/11974 [==============================] - 0s 27us/step - loss: 0.0114 - accuracy: 0.9958 - val_loss: 0.0218 - val_accuracy: 0.9906
Epoch 444/500
11974/11974 [==============================] - 0s 27us/step - loss: 0.0106 - accuracy:

In [7]:
# import matplotlib.pyplot as plt
# import matplotlib

# matplotlib.rcParams.update({'font.size': 20})
# plt.figure(figsize=(20,16))
# plt.plot(history.history['val_accuracy'], color='b', label="Validation Accuracy")
# plt.plot(history.history['accuracy'], color='r', label="Training Accuracy")

# plt.ylim([0.9,1])
# plt.xlim([0,len(history.history['loss'])])
# plt.title('Model accuracy')

# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Validation', 'Train'])
# plt.savefig('cnn1d-2loop-acc.png')
# # plt.show()

In [8]:
# plt.figure(figsize=(20,16))
# plt.plot(history.history['val_loss'], color='b', label="Validation Loss")
# plt.plot(history.history['loss'], color='r', label="Training Loss")

# plt.ylim([0,0.6])
# plt.xlim([0,len(history.history['loss'])])
# plt.title('Model loss')

# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Validation', 'Train'], loc='upper left')
# plt.savefig('cnn1d-2loop-loss.png')
# # plt.show()

In [9]:
max(history.history['val_accuracy'])

0.9935697317123413

In [10]:
model = load_model('../trained_models/cnn1d-2loop2.h5')

datatest = pd.read_csv("../datatest")
x_columns = datatest.columns.drop('label')
x = datatest[x_columns]
x = x.values
y = datatest['label']

x[:, 0] = le_proto.transform(x[:, 0])
#joblib.dump(le_proto, '../preproc/le_proto2.joblib',protocol=1) #save label encoder weights to file
y = le_label.transform(y)
#joblib.dump(le_label, '../preproc/le_label2.joblib',protocol=1) #save label encoder weights to file
x=sc.transform(x)
#joblib.dump(sc, '../preproc/scaler2.joblib',protocol=1)
x = np.expand_dims(x, axis=2)

y_pred = model.predict_classes(x)
y_pred = le_label.inverse_transform(y_pred)
y_test = le_label.inverse_transform(y)
stats = {}
labels = ['normal','tcpsynflood','icmpflood','udpflood','ipsweep','portscan','pingofdeath']
for i in labels:
    stats[i]={}
    for j in labels:
        stats[i][j] = 0


In [11]:
for i in range(len(y)):
    stats[y_test[i]][y_pred[i]] += 1
stats

{'normal': {'normal': 1236,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 2,
  'ipsweep': 0,
  'portscan': 4,
  'pingofdeath': 0},
 'tcpsynflood': {'normal': 1,
  'tcpsynflood': 1043,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 1,
  'pingofdeath': 0},
 'icmpflood': {'normal': 5,
  'tcpsynflood': 0,
  'icmpflood': 779,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'udpflood': {'normal': 2,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 819,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'ipsweep': {'normal': 1,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 646,
  'portscan': 0,
  'pingofdeath': 0},
 'portscan': {'normal': 4,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 914,
  'pingofdeath': 0},
 'pingofdeath': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 461}}

In [12]:
tpr={}
fpr={}
precision={}
f1score={}
for i in labels:
    tp = stats[i][i]
    fp = 0
    tn = 0
    fn = 0
    for j in labels:
        if i != j:
            fn += stats[i][j]
            fp += stats[j][i]
            for k in stats[j].keys():
                if k != i:
                    tn += stats[j][k]
    tpr[i] = tp / (tp + fn)
    fpr[i] = fp / (fp + tn)
    precision[i] = tp / (tp + fp)
    f1score[i] = 2/(1/tpr[i] + 1/precision[i])
print('TPR: \n ' + str(tpr))
print('FRP:\n ' + str(fpr))
print('Precision:\n ' + str(precision))
print('F1-score: \n' + str(f1score))

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

TPR: 
 {'normal': 0.9951690821256038, 'tcpsynflood': 0.9980861244019139, 'icmpflood': 0.9936224489795918, 'udpflood': 0.997563946406821, 'ipsweep': 0.9984544049459042, 'portscan': 0.9956427015250545, 'pingofdeath': 1.0}
FRP:
 {'normal': 0.002780153977758768, 'tcpsynflood': 0.0, 'icmpflood': 0.0, 'udpflood': 0.00039238767902687857, 'ipsweep': 0.0, 'portscan': 0.001, 'pingofdeath': 0.0}
Precision:
 {'normal': 0.9895916733386709, 'tcpsynflood': 1.0, 'icmpflood': 1.0, 'udpflood': 0.997563946406821, 'ipsweep': 1.0, 'portscan': 0.9945593035908596, 'pingofdeath': 1.0}
F1-score: 
{'normal': 0.9923725411481332, 'tcpsynflood': 0.9990421455938697, 'icmpflood': 0.9968010236724248, 'udpflood': 0.997563946406821, 'ipsweep': 0.9992266047950502, 'portscan': 0.995100707675558, 'pingofdeath': 1.0}
Accuracy: 0.9966204798918553
